# FARM Building Blocks

Welcome to the FARM building blocks tutorial! There are many different ways to make use of this repository, but in this notebook, we will be going through the most import building blocks that will help you harvest the rewards of a successfully trained NLP model.

Happy FARMing!

## 1) Text Classification

GermEval 2018 (GermEval2018) (https://projects.fzai.h-da.de/iggsa/) is an open data set containing texts that need to be classified by whether they are offensive or not. There are a set of coarse and fine labels, but here we will only be looking at the coarse set which labels each example as either OFFENSE or OTHER. To tackle this task, we are going to build a classifier that is composed of Google's BERT language model and a feed forward neural network prediction head.

### Setup

In [2]:
# Let's start by adjust the working directory so that it is the root of the repository
# This should be run just once.

import os
os.chdir('../')
print("Current working directory is {}".format(os.getcwd()))

Current working directory is /content/FARM


In [3]:
# Here are the imports we need
import torch
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import LanguageModel
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer
from farm.utils import MLFlowLogger

05/05/2020 08:19:03 - INFO - transformers.file_utils -   PyTorch version 1.4.0 available.
05/05/2020 08:19:04 - INFO - transformers.file_utils -   TensorFlow version 2.2.0-rc3 available.


In [4]:
# Farm allows simple logging of many parameters & metrics. Let's use MLflow framework to track our experiment ...
ml_logger = MLFlowLogger(tracking_uri="https://public-mlflow.deepset.ai/")
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab")


 __          __  _                            _        
 \ \        / / | |                          | |       
  \ \  /\  / /__| | ___ ___  _ __ ___   ___  | |_ ___  
   \ \/  \/ / _ \ |/ __/ _ \| '_ ` _ \ / _ \ | __/ _ \ 
    \  /\  /  __/ | (_| (_) | | | | | |  __/ | || (_) |
     \/  \/ \___|_|\___\___/|_| |_| |_|\___|  \__\___/ 
  ______      _____  __  __  
 |  ____/\   |  __ \|  \/  |              _.-^-._    .--.
 | |__ /  \  | |__) | \  / |           .-'   _   '-. |__|
 |  __/ /\ \ |  _  /| |\/| |          /     |_|     \|  |
 | | / ____ \| | \ \| |  | |         /               \  |
 |_|/_/    \_\_|  \_\_|  |_|        /|     _____     |\ |
                                     |    |==|==|    |  |
|---||---|---|---|---|---|---|---|---|    |--|--|    |  |
|---||---|---|---|---|---|---|---|---|    |==|==|    |  |
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 


In [5]:
# We need to fetch the right device to drive the growth of our model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Devices available: {}".format(device))

Devices available: cuda


### Data Handling

In [6]:
# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

tokenizer = Tokenizer.load(
    pretrained_model_name_or_path="bert-base-german-cased",
    do_lower_case=False)

05/05/2020 08:19:14 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizer'
05/05/2020 08:19:15 - INFO - transformers.tokenization_utils -   loading file https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased-vocab.txt from cache at /root/.cache/torch/transformers/da299cdd121a3d71e1626f2908dda0d02658f42e925a3d6abd8273ec08cf41a6.31ccc255fc2bad3578089a3997f16b286498ba78c0adc43b5bb2a3f9a0d2c85c


In [0]:
# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor

LABEL_LIST = ["OTHER", "OFFENSE"]
processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir="data/germeval18",
                                        label_list=LABEL_LIST,
                                        metric="f1_macro",
                                        label_column_name="coarse_label")

In [8]:
# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

05/05/2020 08:19:21 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
05/05/2020 08:19:21 - INFO - farm.data_handler.data_silo -   Loading train set from: data/germeval18/train.tsv 
05/05/2020 08:19:21 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 5009 dictionaries to pytorch datasets (chunksize = 1002)...
05/05/2020 08:19:21 - INFO - farm.data_handler.data_silo -    0 
05/05/2020 08:19:21 - INFO - farm.data_handler.data_silo -   /|\
05/05/2020 08:19:21 - INFO - farm.data_handler.data_silo -   /'\
05/05/2020 08:19:21 - INFO - farm.data_handler.data_silo -   
Preprocessing Dataset data/germeval18/train.tsv:   0%|          | 0/5009 [00:00<?, ? Dicts/s]05/05/2020 08:19:22 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
05/05/2020 08:19:22 - INFO - farm.data_handler

### Modeling

In FARM, we make a strong distinction between the language model and prediction head so that you can mix and match different building blocks for your needs.

For example, in the transfer learning paradigm, you might have the one language model that you will be using for both document classification and NER. Or you perhaps you have a pretrained language model which you would like to adapt to your domain, then use for a downstream task such as question answering. 

All this is possible within FARM and requires only the replacement of a few modular components, as we shall see below.

Let's first have a look at how we might set up a model.

In [9]:
# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See https://huggingface.co/models for a list of available models

MODEL_NAME_OR_PATH = "bert-base-german-cased"

language_model = LanguageModel.load(MODEL_NAME_OR_PATH)

05/05/2020 08:19:39 - INFO - transformers.modeling_utils -   loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-german-cased-pytorch_model.bin from cache at /root/.cache/torch/transformers/e32f648561b03f77a129832928b7f16decdc5e0870f1e6558857e046169d4133.4e5eda3a0f09b32a0b7d1a9185034da1b3506d5c5b0c6880a7ca0122ab5eef2e
05/05/2020 08:19:41 - INFO - farm.modeling.language_model -   Automatically detected language from language model name: german


In [10]:
# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

prediction_head = TextClassificationHead(num_labels=len(LABEL_LIST))

05/05/2020 08:19:45 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 2]


In [0]:
# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

### Training

In [12]:
# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

LEARNING_RATE = 2e-5
N_EPOCHS = 1

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    device=device,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

05/05/2020 08:20:00 - INFO - farm.modeling.optimization -   Loading optimizer `TransformersAdamW`: '{'correct_bias': False, 'weight_decay': 0.01, 'lr': 2e-05}'
05/05/2020 08:20:01 - INFO - farm.modeling.optimization -   Using scheduler 'get_linear_schedule_with_warmup'
05/05/2020 08:20:01 - INFO - farm.modeling.optimization -   Loading schedule `get_linear_schedule_with_warmup`: '{'num_warmup_steps': 12.600000000000001, 'num_training_steps': 126}'


In [0]:
# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available, to 0 if not
N_GPU = 1

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device,
)

In [14]:
model = trainer.train()

05/05/2020 08:20:09 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 32/32 [00:07<00:00,  4.27it/s]
05/05/2020 08:21:36 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV

In [15]:
# Test your model on a sample (Inference)
from farm.infer import Inferencer
from pprint import PrettyPrinter

infer_model = Inferencer(processor=processor, model=model, task_type="text_classification", gpu=True)

basic_texts = [
    {"text": "Martin ist ein Idiot"},
    {"text": "Martin Müller spielt Handball in Berlin"},
]
result = infer_model.inference_from_dicts(dicts=basic_texts)
PrettyPrinter().pprint(result)

05/05/2020 08:22:39 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
05/05/2020 08:22:39 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
05/05/2020 08:22:39 - INFO - farm.infer -    0 
05/05/2020 08:22:39 - INFO - farm.infer -   /w\
05/05/2020 08:22:39 - INFO - farm.infer -   /'\
05/05/2020 08:22:39 - INFO - farm.infer -   
05/05/2020 08:22:39 - INFO - farm.data_handler.processor -   *** Show 2 random examples ***
05/05/2020 08:22:39 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/_________________________________________

[{'predictions': [{'context': 'Martin ist ein Idiot',
                   'end': None,
                   'label': 'OFFENSE',
                   'probability': 0.907324,
                   'start': None},
                  {'context': 'Martin Müller spielt Handball in Berlin',
                   'end': None,
                   'label': 'OTHER',
                   'probability': 0.9301861,
                   'start': None}],
  'task': 'text_classification'}]


# Switch to NER

In a transfer learning paradigm, there is a core computation that is shared amongst all tasks. FARM's modular structure means that you can easily swap out different building blocks to make the same language model work for many different tasks.

We can adapt the above text classification model to NER by simply switching out the processor and prediction head.

In [0]:
# Import the new building blocks

from farm.data_handler.processor import NERProcessor
from farm.modeling.prediction_head import TokenClassificationHead
ml_logger.init_experiment(experiment_name="Public_FARM", run_name="Tutorial1_Colab_NER")

In [0]:
# This processor will preprocess the data for the CoNLL03 NER task
ner_labels = ["[PAD]", "X", "O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-OTH", "I-OTH"]

ner_processor = NERProcessor(tokenizer=tokenizer, 
                             max_seq_len=128, 
                             data_dir="../data/conll03-de",
                             label_list=ner_labels,
                             metric="seq_f1")

In [18]:
# This prediction head is also a feed forward neural network but expects one
# vector per token in the input sequence and will generate a set of logits
# for each input

ner_prediction_head = TokenClassificationHead(num_labels=len(ner_labels))

05/05/2020 08:23:15 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 13]


In [19]:
# We can integrate these new pieces with the rest using this code
# It is pretty much the same structure as what we had above for text classification

BATCH_SIZE = 32
EMBEDS_DROPOUT_PROB = 0.1
LEARNING_RATE = 2e-5
N_EPOCHS = 1
N_GPU = 1

data_silo = DataSilo(
    processor=ner_processor,
    batch_size=BATCH_SIZE)

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[ner_prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_token"],
    device=device)

model, optimizer, lr_schedule = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS,
    device=device)

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    lr_schedule=lr_schedule,
    device=device,
)

05/05/2020 08:23:32 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
05/05/2020 08:23:32 - INFO - farm.data_handler.data_silo -   Loading train set from: ../data/conll03-de/train.txt 
05/05/2020 08:23:32 - ERROR - farm.data_handler.utils -   Separator 	 for dataset German CONLL03 does not match the requirements. Setting seperator to whitespace
05/05/2020 08:23:32 - INFO - farm.data_handler.utils -    Couldn't find ../data/conll03-de/train.txt locally. Trying to download ...
05/05/2020 08:23:32 - INFO - farm.data_handler.utils -   downloading and extracting file conll03-de to dir /content/data
05/05/2020 08:23:34 - INFO - farm.data_handler.data_silo -   Got ya 1 parallel workers to convert 12152 dictionaries to pytorch datasets (chunksize = 2000)...
05/05/2020 08:23:34 - INFO - farm.data_handler.data_silo -    0 
05/

In [20]:
model = trainer.train()

05/05/2020 08:24:16 - INFO - farm.train -   
 

          &&& &&  & &&             _____                   _             
      && &\/&\|& ()|/ @, &&       / ____|                 (_)            
      &\/(/&/&||/& /_/)_&/_&     | |  __ _ __ _____      ___ _ __   __ _ 
   &() &\/&|()|/&\/ '%" & ()     | | |_ | '__/ _ \ \ /\ / / | '_ \ / _` |
  &_\_&&_\ |& |&&/&__%_/_& &&    | |__| | | | (_) \ V  V /| | | | | (_| |
&&   && & &| &| /& & % ()& /&&    \_____|_|  \___/ \_/\_/ |_|_| |_|\__, |
 ()&_---()&\&\|&&-&&--%---()~                                       __/ |
     &&     \|||                                                   |___/
             |||
             |||
             |||
       , -=-~  .-^- _
              `

Evaluating: 100%|██████████| 90/90 [00:27<00:00,  3.22it/s]
05/05/2020 08:26:06 - INFO - farm.eval -   

\\|//       \\|//      \\|//       \\|//     \\|//
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
***************************************************
***** EV